In [2]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '05p'
per = 3000

data_ = 'TransMut'
encoding_ = 'embedding'

id_ = 'OF_TData_pad14_{}_onlyEmbedding_NOPOS'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
    print('creating dirs')
    os.makedirs(model_folder)
    os.makedirs(result_folder)
else:
    print('dirs exist')


dirs exist


In [3]:
!gpustat

mnemosyne            Wed Jun  8 11:41:43 2022  470.103.01
[0] NVIDIA TITAN V   | 37'C,   0 % |  1381 / 12066 MB | s202357(1377M)
[1] NVIDIA TITAN V   | 41'C,  10 % |  1401 / 12066 MB | s202357(1397M)
[2] NVIDIA TITAN V   | 36'C,   0 % |     0 / 12066 MB |
[3] NVIDIA TITAN V   | 36'C,   0 % |     0 / 12066 MB |
[4] NVIDIA TITAN V   | 34'C,   0 % |     0 / 12066 MB |
[5] NVIDIA TITAN V   | 32'C,   0 % |  2892 / 12066 MB | ansup(1617M) ansup(1271M)
[6] NVIDIA TITAN V   | 33'C,   0 % |     0 / 12066 MB |


In [4]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [5]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
#from sklearn.metrics import classification_report
#from sklearn.utils import class_weight

import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [7]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [8]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
        acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
            acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df

def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        return self.pep_inputs.shape[0]

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx]

In [9]:

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        #self.pos_emb = PositionalEncoding(d_model)
        #self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        
        #enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        
        return enc_outputs, 0


class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        #self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        #print("\n\nAfter concat:", enc_outputs, "\nsize", np.shape(enc_outputs))
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        #dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = enc_outputs.view(enc_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        #print("len(dec_outputs), len(dec_outputs[0])", len(dec_outputs), len(dec_outputs[0]))
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), 0, 0, 0

In [10]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)



def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


f_mean = lambda l: sum(l)/len(l)


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [11]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [12]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

d_model = 32 
d_k = d_v = 32 

d_ff = 512 
n_layers = 1 
n_heads = 3

batch_size = 1024
epochs = 125
threshold = 0.5

In [13]:
cv = 5
count = 0
index_order = []

for outer in range(cv):
        #print(f"\nk_outer: {outer}")
        #print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            #print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                #print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                #print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


def data_with_loader(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df_list.append(pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0)[0:per]) 
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Fold-{} Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list, data
    gc.collect()
    return pep_inputs, hla_inputs, labels, loader

index_order = 0

In [13]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
#independent_fold_metrics_list, external_fold_metrics_list, ys_independent_fold_dict, ys_external_fold_dict = [], [], {}, {}
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = [] 

time_train_c = time.time()

for fold in tqdm(range(20)): 

        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        #if not os.path.exists(path_saver):
        for epoch in range(1, epochs + 1):
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                torch.save(model.eval().state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format(   round( (time.time()-time_train_f)/60, 2)   ))

        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
print("Total training time: {:6.2f} mins.".format(   round( (time.time()-time_train_c)/60, 2)   ))

  0%|                                                                                                             | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
TRAIN ID: [2, 3, 4] 9000
Fold-0 Label info: Data train = Counter({0: 4595, 1: 4405}) 
VAL ID: 1 3000
Fold-0 Label info: Data val = Counter({0: 1550, 1: 1450}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_05p_onlyEmbedding_NOPOS_d20_layer1_multihead3_MODEL0.pkl
Fold-0****Train (Ep avg): Epoch-1/125 | Loss = 0.6337 | Time = 0.0113 sec
tn = 3180, fp = 1415, fn = 1812, tp = 2593
y_pred: 0 = 4992 | 1 = 4008
y_true: 0 = 4595 | 1 = 4405
auc=0.6947|sensitivity=0.5886|specificity=0.6921|acc=0.6414|mcc=0.2823
precision=0.6470|recall=0.5886|f1=0.6164|aupr=0.6860
Fold-0 ****Test  Epoch-1/125: Loss = 0.613501
tn = 1176, fp = 374, fn = 515, tp = 935
y_pred: 0 = 1691 | 1 = 1309
y_true: 0 = 1550 | 1 = 1450
auc=0.7805|sensitivity=0.6448|specificity=0.7587|acc=0.7037|mcc=0.4066
precision=0.7143|recall=0.6448|f1=0.6778|aupr=0.7587
Fold time:   0.00 mins.
Fold-0***

Fold-0****Train (Ep avg): Epoch-15/125 | Loss = 0.1082 | Time = 0.0047 sec
tn = 4433, fp = 162, fn = 195, tp = 4210
y_pred: 0 = 4628 | 1 = 4372
y_true: 0 = 4595 | 1 = 4405
auc=0.9932|sensitivity=0.9557|specificity=0.9647|acc=0.9603|mcc=0.9206
precision=0.9629|recall=0.9557|f1=0.9593|aupr=0.9928
Fold-0 ****Test  Epoch-15/125: Loss = 0.572236
tn = 1352, fp = 198, fn = 331, tp = 1119
y_pred: 0 = 1683 | 1 = 1317
y_true: 0 = 1550 | 1 = 1450
auc=0.9010|sensitivity=0.7717|specificity=0.8723|acc=0.8237|mcc=0.6485
precision=0.8497|recall=0.7717|f1=0.8088|aupr=0.8806
Fold time:   0.00 mins.
Fold-0****Train (Ep avg): Epoch-16/125 | Loss = 0.1016 | Time = 0.0046 sec
tn = 4412, fp = 183, fn = 165, tp = 4240
y_pred: 0 = 4577 | 1 = 4423
y_true: 0 = 4595 | 1 = 4405
auc=0.9942|sensitivity=0.9625|specificity=0.9602|acc=0.9613|mcc=0.9226
precision=0.9586|recall=0.9625|f1=0.9606|aupr=0.9941
Fold-0 ****Test  Epoch-16/125: Loss = 0.556316
tn = 1290, fp = 260, fn = 269, tp = 1181
y_pred: 0 = 1559 | 1 = 1441


Fold-0****Train (Ep avg): Epoch-30/125 | Loss = 0.0060 | Time = 0.0062 sec
tn = 4595, fp = 0, fn = 0, tp = 4405
y_pred: 0 = 4595 | 1 = 4405
y_true: 0 = 4595 | 1 = 4405
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-0 ****Test  Epoch-30/125: Loss = 0.527924
tn = 1320, fp = 230, fn = 191, tp = 1259
y_pred: 0 = 1511 | 1 = 1489
y_true: 0 = 1550 | 1 = 1450
auc=0.9343|sensitivity=0.8683|specificity=0.8516|acc=0.8597|mcc=0.7195
precision=0.8455|recall=0.8683|f1=0.8568|aupr=0.9284
Fold time:   0.00 mins.
Fold-0****Train (Ep avg): Epoch-31/125 | Loss = 0.0048 | Time = 0.0058 sec
tn = 4595, fp = 0, fn = 0, tp = 4405
y_pred: 0 = 4595 | 1 = 4405
y_true: 0 = 4595 | 1 = 4405
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-0 ****Test  Epoch-31/125: Loss = 0.540251
tn = 1321, fp = 229, fn = 192, tp = 1258
y_pred: 0 = 1513 | 1 = 1487
y_true: 

Fold-0 ****Test  Epoch-44/125: Loss = 0.624437
tn = 1332, fp = 218, fn = 193, tp = 1257
y_pred: 0 = 1525 | 1 = 1475
y_true: 0 = 1550 | 1 = 1450
auc=0.9335|sensitivity=0.8669|specificity=0.8594|acc=0.8630|mcc=0.7259
precision=0.8522|recall=0.8669|f1=0.8595|aupr=0.9276
Fold time:   0.00 mins.
Fold-0****Train (Ep avg): Epoch-45/125 | Loss = 0.0011 | Time = 0.0047 sec
tn = 4595, fp = 0, fn = 0, tp = 4405
y_pred: 0 = 4595 | 1 = 4405
y_true: 0 = 4595 | 1 = 4405
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-0 ****Test  Epoch-45/125: Loss = 0.627436
tn = 1336, fp = 214, fn = 195, tp = 1255
y_pred: 0 = 1531 | 1 = 1469
y_true: 0 = 1550 | 1 = 1450
auc=0.9336|sensitivity=0.8655|specificity=0.8619|acc=0.8637|mcc=0.7272
precision=0.8543|recall=0.8655|f1=0.8599|aupr=0.9277
Fold time:   0.00 mins.
Fold-0****Train (Ep avg): Epoch-46/125 | Loss = 0.0010 | Time = 0.0064 sec
tn = 4595, fp = 0, fn = 0, tp = 4405
y_pred: 0 = 

...

Fold-19****Train (Ep avg): Epoch-85/125 | Loss = 0.0002 | Time = 0.0062 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-85/125: Loss = 0.788657
tn = 1303, fp = 235, fn = 192, tp = 1270
y_pred: 0 = 1495 | 1 = 1505
y_true: 0 = 1538 | 1 = 1462
auc=0.9261|sensitivity=0.8687|specificity=0.8472|acc=0.8577|mcc=0.7157
precision=0.8439|recall=0.8687|f1=0.8561|aupr=0.9152
Fold time:   0.00 mins.
Fold-19****Train (Ep avg): Epoch-86/125 | Loss = 0.0002 | Time = 0.0067 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-86/125: Loss = 0.789984
tn = 1304, fp = 234, fn = 192, tp = 1270
y_pred: 0 = 1496 | 1 = 1504
y_tr

Fold-19****Train (Ep avg): Epoch-99/125 | Loss = 0.0001 | Time = 0.0060 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-99/125: Loss = 0.812405
tn = 1304, fp = 234, fn = 192, tp = 1270
y_pred: 0 = 1496 | 1 = 1504
y_true: 0 = 1538 | 1 = 1462
auc=0.9260|sensitivity=0.8687|specificity=0.8479|acc=0.8580|mcc=0.7163
precision=0.8444|recall=0.8687|f1=0.8564|aupr=0.9146
Fold time:   0.00 mins.
Fold-19****Train (Ep avg): Epoch-100/125 | Loss = 0.0001 | Time = 0.0058 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-100/125: Loss = 0.813957
tn = 1304, fp = 234, fn = 192, tp = 1270
y_pred: 0 = 1496 | 1 = 1504
y_

Fold-19****Train (Ep avg): Epoch-114/125 | Loss = 0.0001 | Time = 0.0057 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-114/125: Loss = 0.834343
tn = 1305, fp = 233, fn = 193, tp = 1269
y_pred: 0 = 1498 | 1 = 1502
y_true: 0 = 1538 | 1 = 1462
auc=0.9258|sensitivity=0.8680|specificity=0.8485|acc=0.8580|mcc=0.7163
precision=0.8449|recall=0.8680|f1=0.8563|aupr=0.9138
Fold time:   0.00 mins.
Fold-19****Train (Ep avg): Epoch-115/125 | Loss = 0.0001 | Time = 0.0053 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-115/125: Loss = 0.835910
tn = 1305, fp = 233, fn = 193, tp = 1269
y_pred: 0 = 1498 | 1 = 1502


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [10:19<00:00, 30.97s/it]

Fold-19****Train (Ep avg): Epoch-125/125 | Loss = 0.0001 | Time = 0.0048 sec
tn = 4526, fp = 0, fn = 0, tp = 4474
y_pred: 0 = 4526 | 1 = 4474
y_true: 0 = 4526 | 1 = 4474
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000
Fold-19 ****Test  Epoch-125/125: Loss = 0.849100
tn = 1304, fp = 234, fn = 192, tp = 1270
y_pred: 0 = 1496 | 1 = 1504
y_true: 0 = 1538 | 1 = 1462
auc=0.9258|sensitivity=0.8687|specificity=0.8479|acc=0.8580|mcc=0.7163
precision=0.8444|recall=0.8687|f1=0.8564|aupr=0.9144
Fold time:   0.00 mins.
best_epoch: 40
-----Optimization Finished!-----
Total training time:  10.32 mins.
